In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os 
while 'notebooks' in os.getcwd():
    os.chdir("..")
    
from sklearnex import patch_sklearn
patch_sklearn()
from typing import List
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import csv
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from scipy import stats
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score
from IPython.display import clear_output


import warnings
warnings.filterwarnings("ignore")
from src.constants import CLEANED_HOUSEHOLD_POWER_PATH, CLEANED_WEATHER_PATH, \
    HOUSEHOLD_WEATHER_MERGED_PATH

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Hourly distribution

In [4]:
df = pd.read_pickle(
    HOUSEHOLD_WEATHER_MERGED_PATH    
)

house_consumpion_columns = [
    'global_active_power', 
    'global_reactive_power', 
    'voltage',
    'global_intensity', 
    'sub_metering_1', 
    'sub_metering_2',
    'sub_metering_3',
    'daily_energy_consumption_kwh'
]

df.head()

,global_active_power,global_reactive_power,voltage,global_intensity,sub_metering_1,sub_metering_2,sub_metering_3,numer_sta,pmer,tend,...,phenspe1,phenspe2,phenspe3,nnuage1,ctype1,hnuage1,Nom,Latitude,Longitude,Altitude
date,,,,,,,,,,,,,,,,,,,,,
2007-01-01 00:00:00,2.580,0.136,241.97,10.6,0.0,0.0,0.0,7149.0,101960.0,-70.0,...,1017.0,710.0,1118.0,2.0,8.0,780.0,ORLY,48.716833,2.384333,89.0
2007-01-01 00:01:00,2.552,0.100,241.75,10.4,0.0,0.0,0.0,7149.0,101960.0,-70.0,...,1017.0,710.0,1118.0,2.0,8.0,780.0,ORLY,48.716833,2.384333,89.0
2007-01-01 00:02:00,2.550,0.100,241.64,10.4,0.0,0.0,0.0,7149.0,101960.0,-70.0,...,1017.0,710.0,1118.0,2.0,8.0,780.0,ORLY,48.716833,2.384333,89.0
2007-01-01 00:03:00,2.550,0.100,241.71,10.4,0.0,0.0,0.0,7149.0,101960.0,-70.0,...,1017.0,710.0,1118.0,2.0,8.0,780.0,ORLY,48.716833,2.384333,89.0
2007-01-01 00:04:00,2.554,0.100,241.98,10.4,0.0,0.0,0.0,7149.0,101960.0,-70.0,...,1017.0,710.0,1118.0,2.0,8.0,780.0,ORLY,48.716833,2.384333,89.0


In [5]:
df.columns

Index(['global_active_power', 'global_reactive_power', 'voltage',
       'global_intensity', 'sub_metering_1', 'sub_metering_2',
       'sub_metering_3', 'numer_sta', 'pmer', 'tend', 'cod_tend', 'dd', 'ff',
       't', 'td', 'u', 'vv', 'ww', 'w1', 'w2', 'n', 'nbas', 'hbas', 'cl', 'cm',
       'ch', 'pres', 'rafper', 'per', 'ssfrai', 'perssfrai', 'rr1', 'rr3',
       'phenspe1', 'phenspe2', 'phenspe3', 'nnuage1', 'ctype1', 'hnuage1',
       'Nom', 'Latitude', 'Longitude', 'Altitude'],
      dtype='object')

## Train test split

In [6]:
raw_train_df = df.loc[:'31-12-2009']
raw_test_df = df.loc['2010']

# ------------ train df -------------------------------

raw_train_df['daily_energy_consumption_kwh'] = raw_train_df['global_active_power']\
    .groupby(
        pd.Grouper(freq='1D')
    ).sum()/(60)

train_df = raw_train_df.dropna()

# ------------ test df ----------------------------------

raw_test_df['daily_energy_consumption_kwh'] = raw_test_df['global_active_power']\
    .groupby(
        pd.Grouper(freq='1D')
    ).sum()/(60)

test_df = raw_test_df.dropna()


In [10]:
train_df.index

DatetimeIndex(['2007-01-01', '2007-01-02', '2007-01-03', '2007-01-04',
               '2007-01-05', '2007-01-06', '2007-01-07', '2007-01-08',
               '2007-01-09', '2007-01-10',
               ...
               '2009-12-22', '2009-12-23', '2009-12-24', '2009-12-25',
               '2009-12-26', '2009-12-27', '2009-12-28', '2009-12-29',
               '2009-12-30', '2009-12-31'],
              dtype='datetime64[ns]', name='date', length=1096, freq=None)

In [12]:
validation = train_df.loc['2009':]
train = train_df.loc[:'2008']

In [13]:
kmeans = KMeans(2)


,global_active_power,global_reactive_power,voltage,global_intensity,sub_metering_1,sub_metering_2,sub_metering_3,numer_sta,pmer,tend,...,phenspe2,phenspe3,nnuage1,ctype1,hnuage1,Nom,Latitude,Longitude,Altitude,daily_energy_consumption_kwh
date,,,,,,,,,,,,,,,,,,,,,
2007-01-01,2.580,0.136,241.97,10.6,0.0,0.0,0.0,7149.0,101960.0,-70.0,...,710.0,1118.0,2.000000,8.000000,780.000000,ORLY,48.716833,2.384333,89.0,45.816733
2007-01-02,0.326,0.128,243.15,1.4,0.0,0.0,0.0,7149.0,102090.0,430.0,...,1118.0,3100.0,1.000000,8.000000,480.000000,ORLY,48.716833,2.384333,89.0,29.508733
2007-01-03,0.226,0.000,243.19,1.0,0.0,0.0,0.0,7149.0,103310.0,130.0,...,1106.0,3100.0,1.000000,6.000000,480.000000,ORLY,48.716833,2.384333,89.0,8.626067
2007-01-04,3.480,0.238,239.53,14.4,2.0,1.0,18.0,7149.0,102060.0,-310.0,...,1114.0,3100.0,4.000000,6.000000,240.000000,ORLY,48.716833,2.384333,89.0,43.155117
2007-01-05,0.340,0.140,237.32,1.6,0.0,0.0,0.0,7149.0,102360.0,120.0,...,1106.0,3100.0,1.000000,6.000000,750.000000,ORLY,48.716833,2.384333,89.0,26.880067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-27,0.996,0.066,244.81,4.0,0.0,0.0,0.0,7149.0,103090.0,-20.0,...,1108.0,3100.0,1.363636,7.272727,564.545455,ORLY,48.716833,2.384333,89.0,48.953200
2008-12-28,2.154,0.252,245.47,8.8,0.0,0.0,0.0,7149.0,102850.0,-80.0,...,1108.0,3100.0,1.000000,5.437500,1196.250000,ORLY,48.716833,2.384333,89.0,26.177367
2008-12-29,1.320,0.048,245.09,5.4,0.0,0.0,19.0,7149.0,102580.0,0.0,...,1105.0,3100.0,1.000000,3.937500,2306.250000,ORLY,48.716833,2.384333,89.0,9.254633
